In [1]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from psmiles import PolymerSmiles as PS
from sklearn.metrics  import  mean_squared_error
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F 
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
from optuna.trial import TrialState 

random_seed = 123
torch.manual_seed(random_seed)
np.random.seed(random_seed)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
df = pd.read_pickle("../../data/updated_polymers.pth")

In [3]:
scalar = MinMaxScaler()
data = df["fingerprint_circular"]
target = df["Egc"]

#data = data.values.reshape(-1, 1)  # Reshape data

data_train, data_test, target_train, target_test = train_test_split(data, target, test_size=0.9, random_state=123)

# Scaling target variable
target_train = scalar.fit_transform(target_train.values.reshape(-1, 1))
target_test = scalar.transform(target_test.values.reshape(-1, 1))

# Creating tensors from data

#Training Data
data_train_tensor = torch.tensor(data_train.reset_index(drop = True), dtype=torch.float32)
target_train_tensor = torch.tensor(target_train, dtype=torch.float32)

train_dataset = TensorDataset(data_train_tensor, target_train_tensor)
train_loader = DataLoader(train_dataset, batch_size= 32, shuffle= True)

#Testing Data

data_test_tensor = torch.tensor(data_test.reset_index(drop= True), dtype=torch.float32)
target_test_tensor = torch.tensor(target_test, dtype=torch.float32)

test_dataset = TensorDataset(data_test_tensor, target_test_tensor)
test_loader = DataLoader(test_dataset, shuffle= False)

/tmp/ipykernel_3189133/2487804721.py:16: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  data_train_tensor = torch.tensor(data_train.reset_index(drop = True), dtype=torch.float32)


In [4]:
def define_model(trial):
    layers =  nn.ModuleList([
            nn.Sequential(
                nn.Linear(2048, 1888),
                nn.Dropout(0.296708814),
                nn.PReLU()
            ),
            nn.Sequential(
                nn.Linear(1888, 416),
                nn.Dropout(0.103316943),
                nn.PReLU()
            ),
            nn.Sequential(
                nn.Linear(416, 1632),
                nn.Dropout(0.178598433),
                nn.PReLU()
            ),
            
            nn.Linear(1632, 1)
        ])

    return nn.Sequential(*layers)

In [5]:
def objective(trial):

    # Generate the model.
    
    model = define_model(trial)
    state_dict = torch.load('../../models/molecule_circular.pth')
    model.load_state_dict(state_dict, strict=False)
    model = model.to(DEVICE)

    # Generate the optimizers.
    lr = 0.000364567
    optimizer = optim.Adam(model.parameters(), lr=lr)

    EPOCHS  = trial.suggest_int("EPOCHS", 50, 700)

    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0.0
        for batch_idx,(data, target) in enumerate(train_loader):
            data, target = data.to(DEVICE), target.view(-1).to(DEVICE)
            optimizer.zero_grad()
            output = model(data)
            loss = F.mse_loss(output.view(-1), target)
            running_loss += loss.item()
            loss.backward()
            optimizer.step()
        
        epoch_loss = running_loss / batch_idx

        # Validation of the model.
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(test_loader):
                # Limiting validation data.

                data, target = data.to(DEVICE), target.to(DEVICE)
                pred = model(data)
                target_scaled = scalar.inverse_transform(target.cpu().numpy())
                pred_scaled = scalar.inverse_transform(pred.cpu().detach().numpy())

                test_loss = mean_squared_error(target_scaled, pred_scaled)
                val_loss  += test_loss

        avg_val_loss = val_loss/ batch_idx 

        trial.report(avg_val_loss, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return avg_val_loss

In [6]:
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials = 700, n_jobs= 5)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    # Save results to csv file
    df = study.trials_dataframe().drop(['datetime_start', 'datetime_complete', 'duration'], axis=1)  # Exclude columns
    df = df.loc[df['state'] == 'COMPLETE']        # Keep only results that did not prune
    df = df.drop('state', axis=1)                 # Exclude state column
    df = df.sort_values('value')                  # Sort based on accuracy
    df.to_csv('op_no_freezing.csv', index=False)  # Save to csv file

    # Display results in a dataframe
    print("\nOverall Results (ordered by accuracy):\n {}".format(df))

    # Find the most important hyperparameters
    most_important_parameters = optuna.importance.get_param_importances(study, target=None)

    # Display the most important hyperparameters
    print('\nMost important hyperparameters:')
    for key, value in most_important_parameters.items():
        print('  {}:{}{:.2f}%'.format(key, (15-len(key))*' ', value*100))

[I 2023-12-07 18:39:41,595] A new study created in memory with name: no-name-0ee343cf-4b79-477c-92e8-49c05f59046d
[I 2023-12-07 19:13:10,995] Trial 3 finished with value: 0.635863916464294 and parameters: {'EPOCHS': 109}. Best is trial 3 with value: 0.635863916464294.
[I 2023-12-07 19:19:52,893] Trial 4 finished with value: 0.6534568979553684 and parameters: {'EPOCHS': 132}. Best is trial 3 with value: 0.635863916464294.
[I 2023-12-07 19:33:19,265] Trial 2 finished with value: 0.6510737587106317 and parameters: {'EPOCHS': 176}. Best is trial 3 with value: 0.635863916464294.
[I 2023-12-07 19:43:10,237] Trial 6 finished with value: 0.702363377560697 and parameters: {'EPOCHS': 77}. Best is trial 3 with value: 0.635863916464294.
[I 2023-12-07 19:45:56,575] Trial 0 finished with value: 0.6542010889817799 and parameters: {'EPOCHS': 218}. Best is trial 3 with value: 0.635863916464294.
[I 2023-12-07 19:46:38,375] Trial 7 pruned. 
[I 2023-12-07 19:47:30,579] Trial 10 pruned. 
[I 2023-12-07 19:4